In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[:70000])
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll
4,"((, when, she, thought, it, over, afterwards, ...",Carroll


In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df['ADV_count'] = 0
    df['ADJ_count'] = 0
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.at[i, word] += 1
        
        #Count adj and adv
        count_adj = 0
        count_adv = 0
        for token in sentence:
            if token.pos_ == 'ADJ':
                count_adj +=1
            elif token.pos_ == 'ADV':
                count_adv +=1
        
        df.loc[i, 'ADV_count'] = count_adv
        df.loc[i, 'ADJ_count'] = count_adj
            
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500


,sensation,nose,country,doubtful,fellow,object,livery,grey,distant,frog,...,asunder,foresight,distress,elizabeth,pigeon,reception,text_sentence,text_source,ADV_count,ADJ_count
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,3,3
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,7,7
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,1,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"((, when, she, thought, it, over, afterwards, ...",Carroll,11,5


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
#converting to arry mem error
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.4, random_state=42)

train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9902169289663972

Test set score: 0.9343112244897959


In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(2351, 2635) (2351,)
Training set score: 0.9732028923862186

Test set score: 0.9400510204081632


In [9]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.9544874521480221

Test set score: 0.9298469387755102


# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [10]:
word_counts.shape

(3919, 2637)

In [11]:
from sklearn.svm import LinearSVC

SVC = LinearSVC(C=.05, class_weight='balanced')
SVC.fit(X_train, y_train)

print('Training score:', SVC.score(X_train, y_train))
print('\nTest score:', SVC.score(X_test, y_test))

Training score: 0.9825606125053169

Test score: 0.9508928571428571


In [12]:
y_test.value_counts() / y_test.shape[0]

Austen     0.844388
Carroll    0.155612
Name: text_source, dtype: float64

In [13]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [14]:
sense = gutenberg.raw('austen-sense.txt')
sense = re.sub(r'VOLUME \w+', '', sense)
sense = re.sub(r'CHAPTER \w+', '', sense)
sense = text_cleaner(sense)

sense_doc = nlp(sense)

# Group into sentences.
sense_sents = [[sent, "Austen"] for sent in sense_doc.sents]

# doc is quite long, let's cut it down to the same length as Alice.
sense_sents = sense_sents[0:len(alice_sents)]

#build bag of words
sense_sentences = pd.DataFrame(sense_sents)
sense_bow = bow_features(sense_sentences, common_words)
print('done')

Processing row 0
Processing row 500
done


In [15]:
X_sense_test = np.concatenate((X_train[y_train[y_train=='Carroll'].index],
                               sense_bow.drop(['text_sentence','text_source'], 1)), axis=0)
y_sense_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * sense_bow.shape[0])])

In [16]:
SVC.score(X_sense_test, y_sense_test)

0.6559633027522935